# Chapter 10: Sequence Hacking, Hashing and Slicing

This chapter generalises the `Vector2d` class from the previous chapter to the multidimensional case.

`Vector` will behave as a standard Python immutable flat sequence with elements that are floats. By the end of this chapter it will support:

* Basic sequence protocol: `__len__`, `__getitem__`.

* Safe representation of many instances with many items.

* Proper slicing support, producing new `Vector` instances.

* Aggregate hashing taking into account every contained element value.

* Custom formatting language extension.

`Vector` will also implement dynamic attribute access with `__getattr__` as a way of replacing read-only properties of `Vector2d` (not typical of sequence types).


## Vector: A User-Defined Sequence Type

The strategy to implement `Vector` will be to use composition instead of inheritance. The components will be stored in `array` of floats, and the necessary methods will be implemented to make `Vector` behave like an immutable flat sequence (an example of *duck-typing*, which we will return to).

## Vector Take #1: Vector2d Compatible

Best practice for a sequence constructor is to take the data as an iterable argument in the constructor, like all built-in sequence types do.

Note the use of `reprlib` to condense the output of  `__repr__`. This is because multi-dimensional vectors can be arbitrarily high-dimensional, and we want to control the output somehow.

In [23]:
# Example 10-2. vector_v1.py: derived from vector2d_v1.py

from array import array
import reprlib
import math


class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        # Get limited-length representation of self._components
        components = components[components.find("[") : -1]
        class_name = type(self).__name__
        return f"{class_name}({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**Implementation detail:** In `__repr__`, the same result could have been achieved by `reprlib.repr(list(self._components))`. This would be wasteful because it would require copying every item from `self._components` to a `list` just to use the `list` `repr`.

**Note:** Because of its role in debugging, calling `repr()` on an object should **never raise an exception**.

## Protocols and Duck Typing

In the context of OOP, a *protocol* is an informal interface, defined only in documentation and not in code.

For example, in Python the sequence protocol entails just the `__len__` and `__getitem__` methods. Any class that implements those methods with the standard signature and semantics can be used anywhere a sequence is expected. 

Any experienced Python programmer will see, based on these methods, that it *is* a sequence, even if it subclasses something other class. 

> We say it *is* a sequence because it *behaves* like a sequence, and that is what matters. This has become known as *duck-typing*.

Because protocols are informal and unenforced, one can often get away with implementing only parts of a protocol, if one knows the specific context where a class will be used. For example, iteration requires only `__getitem__`, there is no need to provide `__len__`.

## Vector Take #2: A Slicable Sequence

Supporting a sequence protocol is easy if you can delegate to a sequence attribute in your object, like `self._components`.

In [17]:
class Vector:
    # many lines omitted
    # ...
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [18]:
from examples.vector_v1 import Vector

In [19]:
v = Vector([3, 4, 5])
len(v)

3

In [20]:
v[0], v[1]

(3.0, 4.0)

In [21]:
v[1:]

array('d', [4.0, 5.0])

Slicing is supported, but not very well. It would be better if slicing returned a new `Vector` instance and not an `array`. Every built-in sequence type produces a new instance of its own type when sliced.

### How Slicing Works

To illustrate how slicing works, consider the following example:

In [22]:
# Example 10-4. Checking out behavior of __getitem__ and slices

class MySeq:
    def __getitem__(self, index):
        return index


In [23]:
s = MySeq()
s[1]

1

In [24]:
s[1:4]

slice(1, 4, None)

In [25]:
s[1:4:2]

slice(1, 4, 2)

In [26]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [27]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

Note that the notation `1:4` becomes `slice(1, 4, None)`. Also note that the presence of commas produces a `tuple`, that may even hold severel `slice` objects.

`slice` is a built-in type. Note the attributes `start`, `stop`, `step` and `indices`. The latter is a very useful but mostly uknown method.

In [29]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In other words, `slice.indices` gracefully handles missing or negative indices and slices that are longer than the target sequence. 

This produces "normalised" tuples of nonnegative `start`, `stop` and `stride` integers adjusted to fit within the bounds of a sequence of the given length.

In `Vector` we don't need `slice.indices` because we delegate its handling to `self._components`, but it can be a huge time saver in other settings.

### A Slice-Aware `__getitem__`

In [1]:
class Vector:
    # many lines omitted
    # ...
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(self, numbers.Integral):
            return self._components[index]
        else:
            raise TypeError(f"{cls.__name__} indices must be integers")

In [2]:
from examples.vector_v2 import Vector
v = Vector(range(7))
v[2:5]

Vector([2.0, 3.0, 4.0])

In [3]:
# Note that a slice of length == 1 also creates a Vector
v[-1:]

Vector([6.0])

## Vector Take #3: Dynamic Attribute Access

**Note:** This example seems a bit contrived.

In the multi-dimensional case we lose the ability tot access vector components by name, but it may be convenient to be able to access the first few components with shortcut letter such as `x`, `y`, `z` instead of `v[0]`, `v[1]`, `v[2]`.

In `Vector2d` we provided read-only access using the `@property` decorator. Doing the same in `Vector` would be tedious; the `__getattr__` special method provides a better way.

`__getattr__` is invoked by the interpreter when attribute lookup fails. 

In simple terms, given the expression `my_obj.x`, Python checks if the `my_obj` instance has an attribute `x`.

If not, the seach goes to the class (`my_obj.__class__`), and then up the inheritance graph.

If the `x` attribute is not found, the `__getattr__` method defined in the class of `my_obj` is called with `self` and the name of the attribute as a string (e.g. `"x"`).

In [19]:
# Example 10-8. vector_v3_getattr.py

class Vector:
    # many lines omitted
    # ...
    
    shortcut_names = "xyzt"
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = f"{cls.__name__!r} object has no attribute {name!r}"
        raise AttributeError(msg)
            

Note that Python only calls the `__getattr__` method as a fallback, which makes the above implementation behave inappropriately.

See below for an example.

In [30]:
from examples.vector_v3_getattr import Vector

v = Vector(range(5))
v.x

0.0

In [31]:
v.x = 10
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

When we assign `v.x = 10`, the `v` object now has an `x` attribute, so `__getattr__` will no longer be called to retrieve `v.x`.

In the following we implement `__setattr__` to avoid this inconsistency.

In [ ]:
# Example 10-9. vector_v3_setattr.py

class Vector:
    # many lines omitted
    # ...
    
    shortcut_names = "xyzt"
    
    def __setattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            if name in shortcut_names:
                error = "readonly attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ""
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

The `super()` function provides a way to access methods of superclasses dynamically, a necessity in a dynamic language supporting multiple inheritances like Python.

It is used to delegate some task from a method in a subclass to a suitable method in the superclass.

**Takeaway from this example:** Very often, when you implement `__getattr__`, you will need to code `__setattr__` aswell to avoid inconsistencies.

## Vector Take #4: Hashing and a Faster `==`

Earlier it was mentioned that `functools.reduce` is not as popular as before, but the author uses it in this example to apply the xor operator (`^`) to every component of `Vector`. 

The first argument to `reduce()` is a two-argument function, the second argument is an iterable.

In [2]:
import functools

def fn(x, y):
    return x * y

lst = list(range(1, 5))

functools.reduce(fn, lst, 1)

24

When you call `reduce(fn, lst)`, `fn` is applied to the first pair of elements in `lst`, producing a result `r1`.

Next, `fn` is applied to `r1` and the next element of `lst`, and so on until the last element, when a single result `rN` is produced.

When using reduce, it is good practice to provide the third argument `reduce(function, iterable, initialiser)` to prevent a `TypeError` exception. 

The `initialiser` is the value that will be returned if the sequence is empty, so it should be the identity value of the operation.

In [ ]:
# Example 10-12. Part of vector_v4.py

import operator
import functools

class Vector:
    typecode = "d"
    
    # many lines omitted
    # ...
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

Note that it is good practice to keep `__eq__` and `__hash__` close in source code.

Also note the change to `__eq__` compared to earlier versions that just check equality of tuples.

`all` returns `False` as soon as the first inequality is reached, which makes it more efficient than earlier versions. Also, we fail fast if the tuples are not of same length.

**Note that `zip` stops without warning when one of the iterables is exhausted.** Alternatively, one can use `functools.zip_longest` that uses an optional `fillvalue` to fill missing values so that tuples can be generated until the last iterable is exhausted. 

## Vector Take #5: Formatting

Instead of polar coordinates `Vector` will proved spherical coordinates. Accordingly, the custom format suffix `p` is replaced by `h`(ypersphere).

In [ ]:
# Example 10-16. Final vector_v5.py implementation

import math

class Vector:
    typecode = "d"
    
    # many lines omitted
    # ...
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("h"):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = "<{}>"
        else:
            coords = self
            outer_fmt = "({})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(", ".join(components))


In [4]:
import itertools
help(itertools.chain)

Help on class chain in module itertools:

class chain(builtins.object)
 |  chain(*iterables) --> chain object
 |  
 |  Return a chain object whose .__next__() method returns elements from the
 |  first iterable until it is exhausted, then elements from the next
 |  iterable, until all of the iterables are exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_iterable(...) from builtins.type
 |      chain.from_iterable(iterable) --> chain object
 |      
 |      Alternate chain() constructor taking a single iterable argument
 |      that evaluat